In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/paysim1/PS_20174392719_1491204439457_log.csv


In [6]:
!pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285413 sha256=c0e19185100be73f1ee386a7f54602d1cfee71fffdd92eb97200c5b34f0c199c
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [7]:
import pyspark
from pyspark.sql import SparkSession

In [8]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import pyspark.sql.functions as F

In [9]:
spark=SparkSession.builder.appName("test_pyspark").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/13 20:46:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
data=spark.read.csv("/kaggle/input/paysim1/PS_20174392719_1491204439457_log.csv",header=True,inferSchema=True)

In [11]:
#Data Exploration
data.count()

6362620

In [12]:
data.dtypes

[('step', 'int'),
 ('type', 'string'),
 ('amount', 'double'),
 ('nameOrig', 'string'),
 ('oldbalanceOrg', 'double'),
 ('newbalanceOrig', 'double'),
 ('nameDest', 'string'),
 ('oldbalanceDest', 'double'),
 ('newbalanceDest', 'double'),
 ('isFraud', 'int'),
 ('isFlaggedFraud', 'int')]

In [13]:
data.limit(3).toPandas()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0


In [15]:
data.groupBy('isFraud').count().show() #Used Stratified split method

+-------+-------+
|isFraud|  count|
+-------+-------+
|      1|   8213|
|      0|6354407|
+-------+-------+



In [16]:
### check if there is missing values in the data
from pyspark.sql.functions import col,sum
data.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in data.columns)).show()

+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+--------------+
|step|type|amount|nameOrig|oldbalanceOrg|newbalanceOrig|nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+--------------+
|   0|   0|     0|       0|            0|             0|       0|             0|             0|      0|             0|
+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+--------------+



In [17]:
#Understand the Data which belongs to fraudulent
subd = data.filter(data['isFraud'] == 1).groupby('type').count().show()


+--------+-----+
|    type|count|
+--------+-----+
|TRANSFER| 4097|
|CASH_OUT| 4116|
+--------+-----+



In [13]:
data.groupby('isFlaggedFraud').count().show()

+--------------+-------+
|isFlaggedFraud|  count|
+--------------+-------+
|             0|6362604|
|             1|     16|
+--------------+-------+



In [18]:
#check if this is consistent with the decription of the column that more than 200,200 will be conisdered as illegeal
data.filter(data['isFlaggedFraud'] == 1).orderBy('amount').show()

+----+--------+----------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|    amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+----------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
| 586|TRANSFER| 353874.22|C1684585475|    353874.22|     353874.22|C1770418982|           0.0|           0.0|      1|             1|
| 646|TRANSFER| 399045.08| C724693370|1.039904508E7| 1.039904508E7|C1909486199|           0.0|           0.0|      1|             1|
| 279|TRANSFER| 536624.41|C1035541766|    536624.41|     536624.41|C1100697970|           0.0|           0.0|      1|             1|
| 250|TRANSFER|1343002.08|C1100582606|   1343002.08|    1343002.08|C1147517658|           0.0|           0.0|      1|             1|
| 617|TRANSFER|2542664.27| C786455622|   2542664.27|    2542664.27| C

In [19]:
#Check the min and max of the values when it's flagged as fraud
max1 = data.filter(data['isFlaggedFraud'] == 1).agg({"amount": "max"}).collect()[0]
min1 = data.filter(data['isFlaggedFraud'] == 1).agg({"amount": "min"}).collect()[0]

In [20]:
max1 = data.filter(data['isFlaggedFraud'] == 1).agg({"amount": "mean"}).collect()[0]

In [21]:
data.groupBy(['isFlaggedFraud']).agg({'amount':'avg','isFlaggedFraud':'count'}).show()


+--------------+------------------+---------------------+
|isFlaggedFraud|       avg(amount)|count(isFlaggedFraud)|
+--------------+------------------+---------------------+
|             0|179850.13041768773|              6362604|
|             1|    4861597.730625|                   16|
+--------------+------------------+---------------------+



In [22]:
#Multiple columns and multiple functions
from pyspark.sql.functions import mean, sum, max, col,count
groupBy = ["isFlaggedFraud"]
aggregate = ["amount"]
funs = [mean, sum, max,count]

exprs = [f(col(c)) for f in funs for c in aggregate]

In [23]:
re1 = data.groupby(*groupBy).agg(*exprs).collect()

In [24]:
re1

[Row(isFlaggedFraud=0, avg(amount)=179850.13041768773, sum(amount)=1144315159196.1016, max(amount)=92445516.64, count(amount)=6362604),
 Row(isFlaggedFraud=1, avg(amount)=4861597.730625, sum(amount)=77785563.69, max(amount)=10000000.0, count(amount)=16)]

In [25]:
max0 = data.filter(data['isFlaggedFraud'] == 0).agg({"amount": "max"}).collect()[0]
min0 = data.filter(data['isFlaggedFraud'] == 0).agg({"amount": "min"}).collect()[0]


In [26]:
max1[0],min1[0],max0[0],min0[0]


(4861597.730625, 353874.22, 92445516.64, 0.0)

In [27]:
data.filter( (data['isFlaggedFraud'] == 0) & (data['oldbalanceDest'] ==0) & (data['newbalanceDest'] ==0)).show(5)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|  amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1| PAYMENT| 9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1| PAYMENT| 1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|
|   1|TRANSFER|   181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|
|   1| PAYMENT|11668.14|C2048537720|      41554.0|      29885.86|M1230701703|           0.0|           0.0|      0|             0|
|   1| PAYMENT| 7817.71|  C90045638|      53860.0|      46042.29| M573487274|      

In [28]:
### whether the account names is duplicated or not when it is flagged as fraud
data.filter(data['isFlaggedFraud'] == 1).groupBy('nameOrig').count().show()

+-----------+-----+
|   nameOrig|count|
+-----------+-----+
| C728984460|    1|
|C1035541766|    1|
|C1100582606|    1|
|C2140038573|    1|
|C1684585475|    1|
| C452586515|    1|
| C917414431|    1|
| C724693370|    1|
| C992223106|    1|
|C1869569059|    1|
|  C19004745|    1|
| C908544136|    1|
| C689608084|    1|
| C193696150|    1|
| C786455622|    1|
|C1892216157|    1|
+-----------+-----+



In [29]:
data.filter(data['isFlaggedFraud'] == 1).groupBy('nameDest').count().show()

+-----------+-----+
|   nameDest|count|
+-----------+-----+
| C639921569|    1|
|C1147517658|    1|
|C1100697970|    1|
|C1806199534|    1|
|C1861208726|    1|
|C1395467927|    1|
| C661958277|    1|
|C1392803603|    1|
| C484597480|    1|
|C1909486199|    1|
|C1082139865|    1|
|C1770418982|    1|
|C1308068787|    1|
|C1109166882|    1|
| C891140444|    1|
|C1366804249|    1|
+-----------+-----+



In [30]:
subd1 = data.filter( (data['type'] == 'CASH_IN') & \
                (data['nameOrig'].like('%M%')) ).show(4)

+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+--------------+
|step|type|amount|nameOrig|oldbalanceOrg|newbalanceOrig|nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+--------------+
+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+--------------+



In [31]:
subd1 = data.filter( (data['type'] == 'CASH_OUT') & \
                (data['nameOrig'].like('%M%')) ).show(4)

+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+--------------+
|step|type|amount|nameOrig|oldbalanceOrg|newbalanceOrig|nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+--------------+
+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+--------------+



In [32]:
data.filter(data['nameDest'].like('%M%')).groupBy('type').count().show()

+-------+-------+
|   type|  count|
+-------+-------+
|PAYMENT|2151495|
+-------+-------+



In [33]:
data.filter( (col('nameDest').isin(col('nameOrig'))) & (data['isFraud'] == 1)).show(5)

+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+--------------+
|step|type|amount|nameOrig|oldbalanceOrg|newbalanceOrig|nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+--------------+
+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+--------------+



In [34]:
#Names of Fraud and not Fraud
df1 = data.filter(data['isFraud'] == 1) ## fraud
df0 = data.filter(data['isFraud'] == 0) ## not fraud

In [35]:
##Get the unique names if the data is fraud
unique_org_names1 = [i.nameOrig for i in df1.select('nameOrig').distinct().collect()]
unique_org_names0 = [i.nameOrig for i in df0.select('nameOrig').distinct().collect()]

In [36]:
data.filter(col('nameDest').isin(unique_org_names1)).show(5)

+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+--------------+
|step|type|amount|nameOrig|oldbalanceOrg|newbalanceOrig|nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+--------------+
+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+--------------+



In [37]:
#Data Cleaning
x0 = data.filter(data['type'].isin(['CASH_OUT','TRANSFER']))
data.select(['type']).distinct().show()

+--------+
|    type|
+--------+
|TRANSFER|
| CASH_IN|
|CASH_OUT|
| PAYMENT|
|   DEBIT|
+--------+



In [38]:
#Destination account-replace zero with 1
## chang 0 to more sensible values for some cases
from pyspark.sql.functions import when
import numpy as np

x0 = x0.withColumn('oldbalanceDest', \
              when((x0['oldbalanceDest'] == 0) & (x0['newbalanceDest'] ==0) & (x0['amount'] !=0), -1).otherwise(x0['oldbalanceDest']))

x0 = x0.withColumn('newbalanceDest', \
              when((x0['oldbalanceDest'] == 0) & (x0['newbalanceDest'] ==0) & (x0['amount'] !=0), -1).otherwise(x0['newbalanceDest']))

In [39]:
x0.agg({"oldbalanceDest": "min"}).collect()[0]

Row(min(oldbalanceDest)=-1.0)

In [40]:
#Convert string to numeric columns
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
'''x0 is the data frame'''
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(x0) for column in ['type'] ]

pipeline = Pipeline(stages=indexers)
x_r = pipeline.fit(x0).transform(x0)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [41]:
x_r.limit(5).toPandas()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_index
0,1,TRANSFER,181.00,C1305486145,181.0,0.0,C553264065,-1.0,0.00,1,0,1.0
1,1,CASH_OUT,181.00,C840083671,181.0,0.0,C38997010,21182.0,0.00,1,0,0.0
2,1,CASH_OUT,229133.94,C905080434,15325.0,0.0,C476402209,5083.0,51513.44,0,0,0.0
3,1,TRANSFER,215310.30,C1670993182,705.0,0.0,C1100439041,22425.0,0.00,0,0,1.0
4,1,TRANSFER,311685.89,C1984094095,10835.0,0.0,C932583850,6267.0,2719172.89,0,0,1.0


In [42]:
from pyspark.sql.types import IntegerType

In [43]:
xx = x_r.withColumn('type_index',x_r['type_index'].cast(IntegerType()))
xx = xx.withColumn('isFraud',x_r['isFraud'].cast(IntegerType()))

In [44]:
xx = xx.withColumnRenamed('isFraud','label')

In [45]:
xx.limit(5).toPandas()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,label,isFlaggedFraud,type_index
0,1,TRANSFER,181.00,C1305486145,181.0,0.0,C553264065,-1.0,0.00,1,0,1
1,1,CASH_OUT,181.00,C840083671,181.0,0.0,C38997010,21182.0,0.00,1,0,0
2,1,CASH_OUT,229133.94,C905080434,15325.0,0.0,C476402209,5083.0,51513.44,0,0,0
3,1,TRANSFER,215310.30,C1670993182,705.0,0.0,C1100439041,22425.0,0.00,0,0,1
4,1,TRANSFER,311685.89,C1984094095,10835.0,0.0,C932583850,6267.0,2719172.89,0,0,1


In [46]:
xf = xx.drop('type')

In [47]:
xf.limit(5).toPandas()

,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,label,isFlaggedFraud,type_index
0,1,181.00,C1305486145,181.0,0.0,C553264065,-1.0,0.00,1,0,1
1,1,181.00,C840083671,181.0,0.0,C38997010,21182.0,0.00,1,0,0
2,1,229133.94,C905080434,15325.0,0.0,C476402209,5083.0,51513.44,0,0,0
3,1,215310.30,C1670993182,705.0,0.0,C1100439041,22425.0,0.00,0,0,1
4,1,311685.89,C1984094095,10835.0,0.0,C932583850,6267.0,2719172.89,0,0,1


In [48]:
#Feature Engineering
from pyspark.sql.functions import col


In [49]:
xf = xf.withColumn("errorbalanceOrig", col("newbalanceOrig") + col("amount") - col("oldbalanceOrg"))


In [50]:
xf = xf.withColumn("errorbalanceDest", col("oldbalanceDest") + col("amount") - col("newbalanceDest"))

In [51]:

xf.limit(5).toPandas()

,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,label,isFlaggedFraud,type_index,errorbalanceOrig,errorbalanceDest
0,1,181.00,C1305486145,181.0,0.0,C553264065,-1.0,0.00,1,0,1,0.00,180.0
1,1,181.00,C840083671,181.0,0.0,C38997010,21182.0,0.00,1,0,0,0.00,21363.0
2,1,229133.94,C905080434,15325.0,0.0,C476402209,5083.0,51513.44,0,0,0,213808.94,182703.5
3,1,215310.30,C1670993182,705.0,0.0,C1100439041,22425.0,0.00,0,0,1,214605.30,237735.3
4,1,311685.89,C1984094095,10835.0,0.0,C932583850,6267.0,2719172.89,0,0,1,300850.89,-2401220.0


In [52]:
yv = ['isFraud']
xv = ['step',
      'type_index',
      'amount','oldbalanceOrg','newbalanceOrig',
      'oldbalanceDest','newbalanceDest',"errorbalanceOrig","errorbalanceDest"]

In [53]:
xf.dtypes

[('step', 'int'),
 ('amount', 'double'),
 ('nameOrig', 'string'),
 ('oldbalanceOrg', 'double'),
 ('newbalanceOrig', 'double'),
 ('nameDest', 'string'),
 ('oldbalanceDest', 'double'),
 ('newbalanceDest', 'double'),
 ('label', 'int'),
 ('isFlaggedFraud', 'int'),
 ('type_index', 'int'),
 ('errorbalanceOrig', 'double'),
 ('errorbalanceDest', 'double')]

In [54]:
sxv = ['step',
      'amount','oldbalanceOrg','newbalanceOrig',
      'oldbalanceDest','newbalanceDest']
for col_name in sxv:
    xf = xf.withColumn(col_name, col(col_name).cast('float'))


In [55]:

xf.dtypes

[('step', 'float'),
 ('amount', 'float'),
 ('nameOrig', 'string'),
 ('oldbalanceOrg', 'float'),
 ('newbalanceOrig', 'float'),
 ('nameDest', 'string'),
 ('oldbalanceDest', 'float'),
 ('newbalanceDest', 'float'),
 ('label', 'int'),
 ('isFlaggedFraud', 'int'),
 ('type_index', 'int'),
 ('errorbalanceOrig', 'double'),
 ('errorbalanceDest', 'double')]

In [56]:
#Train and test data split
from pyspark.sql.functions import lit

In [64]:
fractions = xf.select("label").distinct().withColumn("fraction", lit(0.8)).rdd.collectAsMap()
df_train = xf.sampleBy('label',fractions,seed=17)
df_test = xf.subtract(df_train)

In [65]:
#Model-Pipeline
yv = ['label']
xv = ['step','type_index','amount','oldbalanceOrg','newbalanceOrig',
      'oldbalanceDest','newbalanceDest',"errorbalanceOrig","errorbalanceDest"]


In [66]:
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols= xv, outputCol="va")

In [67]:
#Convert dataframe to vector
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator,  TrainValidationSplit, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [68]:
#Creating PCA
from pyspark.ml.feature import PCA

pca = PCA(k=5, inputCol="va", outputCol="features")

In [74]:
df_train.show()

+----+---------+-----------+-------------+--------------+-----------+--------------+--------------+-----+--------------+----------+--------------------+-------------------+
|step|   amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|label|isFlaggedFraud|type_index|    errorbalanceOrig|   errorbalanceDest|
+----+---------+-----------+-------------+--------------+-----------+--------------+--------------+-----+--------------+----------+--------------------+-------------------+
| 1.0|    181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|    1|             0|         0|                 0.0|            21363.0|
| 1.0|229133.94| C905080434|      15325.0|           0.0| C476402209|        5083.0|      51513.44|    0|             0|         0|           213808.94|           182703.5|
| 1.0| 215310.3|C1670993182|        705.0|           0.0|C1100439041|       22425.0|           0.0|    0|             0|         1|    

In [ ]:
#RF classifer
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="label", featuresCol="features")



In [ ]:
#Creating a Multiclass Classification Evaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator()

In [ ]:
#Creating a Pipeline
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[vecAssembler, pca, rf])

In [ ]:
#Hyperparameter tunning
#creating Grid search Configuration
from pyspark.ml.tuning import CrossValidator, TrainValidationSplit, ParamGridBuilder

paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [2, 4, 6])
             .addGrid(rf.maxBins, [20, 60])
             .addGrid(rf.numTrees, [5, 20])
             .addGrid(rf.impurity, ['gini'])
             .build())

def get_validation(by='cv'):
    if by is 'cv': ##cross validation
        return CrossValidator(
            estimator=pipeline,
            estimatorParamMaps=paramGrid,
            evaluator=evaluator,
            numFolds=10)

    elif by is 'tvs': ##train and test split
        return TrainValidationSplit(
            estimator=pipeline,
            estimatorParamMaps=paramGrid,
            evaluator=evaluator,
            # 80% of the data will be used for training, 20% for validation.
            trainRatio=0.8)
    else:
        print('please, pick either cv or tvs')
        return None

def evaluate(model, df_train, df_test):
    train_pred = model.transform(df_train)
    train_acc = evaluator.evaluate(train_pred)

    test_pred = model.transform(df_test)
    test_acc = evaluator.evaluate(test_pred)

    print('train_acc: {}, test_acc: {}'.format(train_acc, test_acc))

    return train_pred, test_pred

In [ ]:

df_train.dtypes

In [ ]:
val = get_validation('cv')
model_cv = val.fit(df_train)


In [ ]:
bestModel = model_cv.bestModel
finalPredictions = bestModel.transform(df_train)
evaluator.evaluate(finalPredictions)

In [ ]:
#hyperparameter of the best model
bestModel.stages[-1]._java_obj.getImpurity()

In [ ]:
bestModel.stages[-1]._java_obj.getMaxDepth()


In [ ]:
bestModel.stages[-1]._java_obj.getMaxBins()

In [ ]:
bestModel.stages[-1]._java_obj.getNumTrees()


In [ ]:
#the paramters are empty when tried the following way
{param[0].name: param[1] for param in bestModel.extractParamMap().items()}

In [ ]:
#Evalution of Model-Accuracy
train_pred, test_pred = evaluate(model_cv, df_train, df_test)

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

class Util:

    def report_classification(model, df_train, df_test, X_features, y_feature):

        classes_train = np.unique(df_train[y_feature].values).tolist()
        classes_test = np.unique(df_test[y_feature].values).tolist()

        assert (classes_train == classes_test)

        classes = classes_train # The order of class is important!

        X_train = df_train[X_features].values.tolist()
        X_test = df_test[X_features].values.tolist()

        y_train = df_train[y_feature].values.tolist()
        y_test = df_test[y_feature].values.tolist()

        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        report_cm(y_train, y_test, y_train_pred, y_test_pred, classes)

    def report_cm(y_train, y_test, y_train_pred, y_test_pred, classes):
        figure, axes = plt.subplots(1, 2, figsize=(10,5))

        cm_test = confusion_matrix(y_test, y_test_pred)
        df_cm_test = pd.DataFrame(cm_test, index = classes, columns = classes)
        ax = sns.heatmap(df_cm_test, annot=True, ax = axes[0], square= True)
        ax.set_title('Test CM')

        cm_train = confusion_matrix(y_train, y_train_pred)
        df_cm_train = pd.DataFrame(cm_train, index = classes, columns = classes)
        ax = sns.heatmap(df_cm_train, annot=True, ax = axes[1], square= True)
        ax.set_title('Train CM')

        print('-' * 20 + 'Testing Performance' + '-' * 20)
        print(classification_report(y_test, y_test_pred, target_names = classes))
        print('acc: ', metrics.accuracy_score(y_test, y_test_pred))

        print('-' * 20 + 'Training Performance' + '-' * 20)
        print(classification_report(y_train, y_train_pred, target_names = classes))
        print('acc: ', metrics.accuracy_score(y_train, y_train_pred))

In [ ]:
y_train = train_pred.select('label').toPandas().apply(lambda x : x[0], 1).values.tolist()
y_train_pred = train_pred.select('prediction').toPandas().apply(lambda x : x[0], 1).values.tolist()

y_test = test_pred.select('label').toPandas().apply(lambda x : x[0], 1).values.tolist()
y_train_test = test_pred.select('prediction').toPandas().apply(lambda x : x[0], 1).values.tolist()

Util.report_cm(y_train, y_test, y_train_pred, y_train_test, ['0', '1'])

In [ ]:
selected = train_pred.select("label", "prediction", "probability")
display(selected.show(10))

In [78]:
!pip install xgboost

In [75]:
!pip install pyspark py4j

In [76]:
from xgboost.spark import SparkXGBClassifier

In [77]:
from pyspark.ml.tuning import ParamGridBuilder

In [78]:
xgboost_classifier = SparkXGBClassifier()

In [80]:
paramGrid = (ParamGridBuilder()
             .addGrid(xgboost_classifier.max_depth, [3, 5, 7])
             .addGrid(xgboost_classifier.learning_rate, [0.01, 0.05, 0.1])
             .build())

In [81]:
#Creating a Multiclass Classification Evaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator()

In [82]:
#Creating a Pipeline
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[vecAssembler, pca, xgboost_classifier])

In [83]:
cv = CrossValidator(
            estimator=pipeline,
            estimatorParamMaps=paramGrid,
            evaluator=evaluator,
            numFolds=10)

In [84]:
def evaluate(model, df_train, df_test):
    train_pred = model.transform(df_train)
    train_acc = evaluator.evaluate(train_pred)

    test_pred = model.transform(df_test)
    test_acc = evaluator.evaluate(test_pred)

    print('train_acc: {}, test_acc: {}'.format(train_acc, test_acc))

    return train_pred, test_pred

In [85]:
model_cv = cv.fit(df_train)

23/08/13 21:50:02 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/08/13 21:50:04 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[21:50:14] task 0 got new rank 0
/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:782: UserWarning: Loading a native XGBoost model with Scikit-Learn interface.
  warnings.warn("Loading a native XGBoost model with Scikit-Learn interface.")
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-pa

In [86]:
bestModel = model_cv.bestModel
finalPredictions = bestModel.transform(df_train)
evaluator.evaluate(finalPredictions)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


0.9990723283344302

In [87]:
evaluate(model_cv, df_train, df_test)

train_acc: 0.9990723283344302, test_acc: 0.9990500622900608


(DataFrame[step: float, amount: float, nameOrig: string, oldbalanceOrg: float, newbalanceOrig: float, nameDest: string, oldbalanceDest: float, newbalanceDest: float, label: int, isFlaggedFraud: int, type_index: int, errorbalanceOrig: double, errorbalanceDest: double, va: vector, features: vector, rawPrediction: vector, prediction: double, probability: vector],
 DataFrame[step: float, amount: float, nameOrig: string, oldbalanceOrg: float, newbalanceOrig: float, nameDest: string, oldbalanceDest: float, newbalanceDest: float, label: int, isFlaggedFraud: int, type_index: int, errorbalanceOrig: double, errorbalanceDest: double, va: vector, features: vector, rawPrediction: vector, prediction: double, probability: vector])